# TP 1 : Agent Réflexe Simple (Robot Aspirateur) 

## Hany El Atlassi 
CSCC - TP 1

## Objectifs du TP 

- Comprendre la boucle Perception → Décision → Action 
- Implémenter un agent réflexe simple (sans mémoire) qui nettoie une grille 3×3 
- Mesurer la performance d'un agent (% de cases propres, nombre d'actions) 
- Analyser le comportement d'un agent autonome 
Vous allez programmer un robot aspirateur qui doit nettoyer une grille 3×3 contenant des cases 
sales et propres. 
Représentation de la grille: 
- 0 = Case propre, 1 = Case sale, R = Position du robot 
### Exemple : 
                                        R 0 1                                               
                                        1 1 0                                                
                                        0 1 1                                                
Le robot doit suivre des règles réflexes simples pour nettoyer efficacement. 

### *PARTIE A : Environnement et Utilitaires*

In [42]:
import random 
def init_grille(n=3, p=0.5, seed=0): 
 """ 
 Initialise une grille n×n avec p% de cases sales. 
 Args: 
 n: Taille de la grille 
 p: Probabilité qu'une case soit sale (0.0 à 1.0) 
 seed: Graine aléatoire pour la reproductibilité  
 Returns: 
 Liste de listes représentant la grille 
 """ 
 rnd = random.Random(seed) 
 return [[1 if rnd.random() < p else 0 for _ in range(n)] for _ in range(n)]

In [43]:
def tout_propre(G): 
    """Vérifie si toutes les cases sont propres.""" 
    return all(c == 0 for row in G for c in row) 
 
def in_bounds(i, j, n): 
    """Vérifie si la position (i,j) est dans la grille n×n.""" 
    return 0 <= i < n and 0 <= j < n 
 
def print_grille(G, pos=None, label=""): 
    """ 
    Affiche la grille. 
    Args: 
        G: La grille à afficher 
        pos: Position du robot (i, j) ou None 
        label: Titre à afficher 
    """ 
    if label:  
        print(label) 
    for i, row in enumerate(G): 
        line = [] 
        for j, c in enumerate(row): 
            if pos is not None and (i, j) == pos: 
                line.append("R") 
            else: 
                line.append(str(c)) 
        print(" ".join(line)) 
    print() 
 

1. Créez une grille 3×3 avec 60% de cases sales  
2. Choisissez une position initiale (x, y) au hasard pour le robot 
3. Affichez la grille avec print_grille 

In [44]:
# TODO: Initialisez les paramètres 
n = 3          # Taille de la grille 
p = 60         # % de saleté 
seed = 42       # Pour la reproductibilité 
# TODO: Créez la grille 
G = init_grille(n=n, p=p, seed=seed) 
# TODO: Position initiale aléatoire du robot 
x, y = (random.randint(0, n-1), random.randint(0, n-1))
# TODO: Affichez la grille 
print_grille(G, pos=(x, y), label="Grille initiale (R = robot) :")

Grille initiale (R = robot) :
1 R 1
1 1 1
1 1 1



### Questions : 
- Q1.1 Quelle est la grille générée avec seed=1 ? Dessinez-la. 
- Q1.2 Combien de cases sales y a-t-il dans cette grille ? 
- Q1.3 Quelle est la position initiale du robot ? 
- Q1.4 Testez avec seed=2 et seed=3. Les grilles changent-elles ? Pourquoi utilise-t-on une graine (seed) ? 
- Q1.5 Que se passe-t-il si vous mettez p=1.0 ? Et p=0.0 ?

In [45]:
n = 3
p = 0.6
seed = 1

seed_one = init_grille(n,p,seed)
random.seed(seed)
(x,y) = (random.randint(0, n-1), random.randint(0, n-1))
print_grille(seed_one, pos=(x, y), label="Grille initiale (R = robot) :")

Grille initiale (R = robot) :
1 0 R
1 1 1
0 0 1



### Réponses
- Q1.2: 5 cases sales
- Q1.3: (0,2)
- Q1.4: Les grilles changent. On utilise une graine spécifique pour fixer une smiluation aléatoire de la grille et de la position du robot.
- Q1.5: Pour p = 1 et p = 0, on crée respectivement une grille remplie de cases sales et une grille remplie de cases propres.

In [46]:
n = 3
p = 0.6
seed = 2
seed_two = init_grille(n,p,seed)
random.seed(seed)
(x,y) = (random.randint(0, n-1), random.randint(0, n-1))
print_grille(seed_two, pos=(x, y), label="Grille initiale (R = robot) :")

Grille initiale (R = robot) :
R 0 1
1 0 0
0 1 0



In [47]:
n = 3
p = 0.6
seed = 3
seed_three = init_grille(n,p,seed)
random.seed(seed)
(x,y) = (random.randint(0, n-1), random.randint(0, n-1))
print_grille(seed_three, pos=(x, y), label="Grille initiale (R = robot) :")

Grille initiale (R = robot) :
1 1 R
0 0 1
1 0 1



In [48]:
n = 3
p = 1
seed = 1
seed_one = init_grille(n,p,seed)
random.seed(seed)
(x,y) = (random.randint(0, n-1), random.randint(0, n-1))
print_grille(seed_one, pos=(x, y), label="Grille initiale (R = robot) :")

Grille initiale (R = robot) :
1 1 R
1 1 1
1 1 1



In [49]:
n = 3
p = 0
seed = 1
seed_one = init_grille(n,p,seed)
random.seed(seed)
(x,y) = (random.randint(0, n-1), random.randint(0, n-1))
print_grille(seed_one, pos=(x, y), label="Grille initiale (R = robot) :")

Grille initiale (R = robot) :
0 0 R
0 0 0
0 0 0



## *Partie B: Règles Réflexes* 

#### Principe de l'agent réflexe : L'agent suit des règles simples  
- **RÈGLE 1** : SI la case courante est sale (=1) 
          ALORS ASPIRER (nettoyer la case) 
 
- **RÈGLE 2** : SINON se déplacer selon un cycle : 
          DROITE → BAS → GAUCHE → HAUT → DROITE → ... 
          (en évitant les bords de la grille)

In [50]:
# Cycle de déplacement 
cycle = ["DROITE", "BAS", "GAUCHE", "HAUT"] 
def choisir_action(G, x, y, step): 
    # Règle 1 : Aspirer si la case est sale 
    if G[x][y] == 1:
        return "ASPIRER"
    # Règle 2 : Suivre le cycle de déplacement 
    for t in range(4): 
        a = cycle[(step + t) % 4] 
        nx, ny = x, y 
        if a == "HAUT":  
            nx -= 1
        if a == "BAS":  
            nx += 1
        if a == "GAUCHE":  
            ny -= 1
        if a == "DROITE":  
            ny += 1
        # Vérifier si la nouvelle position est valide 
        if in_bounds(nx, ny, len(G)): 
            return a 
    return "RIEN" 

**Appliquer une Action**  

Complétez la fonction appliquer_action qui : 
- Si l'action est "ASPIRER" → met la case courante à 0 (propre) 
- Si l'action est un déplacement ("HAUT", "BAS", etc.) → déplace le robot (si possible) 
- Retourne la nouvelle position (x, y) et l'action effectuée

In [51]:
def appliquer_action(G, x, y, action): 
    # TODO: Si l'action est ASPIRER 
    if action == "ASPIRER": 
        # Nettoyez la case courante 
        G[x][y] =  0
        return x, y, action 
     
    # TODO: Si l'action est HAUT et que c'est possible 
    if action == "HAUT" and in_bounds(x-1, y, len(G)): 
        return x-1, y, action
     
    # TODO: Complétez pour BAS 
    if action == "BAS" and in_bounds(x+1, y, len(G)): 
        return x+1, y, action
     
    # TODO: Complétez pour GAUCHE 
    if action == "GAUCHE" and in_bounds(x, y-1, len(G)): 
        return x, y-1, action
     
    # TODO: Complétez pour DROITE 
    if action == "DROITE" and in_bounds(x, y+1, len(G)): 
        return x, y+1, action
     
    # Si aucune action n'est possible 
    return x, y, "RIEN"

### Questions:
- **Q2.1**:  Que fait exactement G[x][y] = 0 ? 
- **Q2.2**:  Pourquoi vérifie-t-on in_bounds avant de se déplacer ? 
- **Q2.3**:  Que se passe-t-il si le robot est dans un coin et essaie d'aller vers le bord ? 
- **Q2.4**:  Testez manuellement : Si le robot est en (0, 0) et fait "HAUT", quelle est la nouvelle position ? 
- **Q2.5**:  Tracez sur papier le parcours du robot pour les 5 premières actions avec la grille de Q1.

### Réponses: 
- **Q2.1**: G[x][y] = 0 réalise l'action "ASPIRER"
- **Q2.2**: On vérifie que le mouvement qu'on essaie de réaliser est possible tout d'abord.
- **Q2.3**: L'action qui essai d'aller vers le bords ne s'executera pas, car in_bounds est une condition d'éxuction de l'action. 
- **Q2.4**: La nouvelle position sera (0,1), car dans le cycle de mouvement du robot, après "HAUT" vient "DROITE". "HAUT" ne s'éxecute pas et passe vers l'action suivante dans le cycle.
- **Q2.5**: Pour la grille suivante:  
La position du robot est (0,2), le cycle est ["DROITE", "BAS", "GAUCHE", "HAUT"]:  
 1. "DROITE" ne s'éxecute pas - (0,2)
 2. "BAS" s'éxecute - (1,2) - la case devient propre (1 => 0)
 3. "GAUCHE" s'éxecute - (1,1) - la case devient propre (1 => 0)
 4. "HAUT" s'éxecute - (0,1) - la case est déja propre (0 => 0)
 5. "DROITE" s'éxecute - (0,2) - la case est déja propre (0 => 0)


In [54]:
n = 3
p = 0.6
seed = 1

seed_one = init_grille(n,p,seed)
random.seed(seed)
(x,y) = (random.randint(0, n-1), random.randint(0, n-1))
print_grille(seed_one, pos=(x, y), label="Grille initiale (R = robot) :")

Grille initiale (R = robot) :
1 0 R
1 1 1
0 0 1



## *Partie C: Boucle d'Execution et Performance*

On va maintenant faire tourner le robot en boucle jusqu'à ce que :  
- Soit toute la grille soit propre 
- Soit on atteigne un nombre maximum d'actions (30 dans notre cas) 

In [58]:
# Paramètres de simulation 
n = 3
p = 0.6
seed = 1
G = init_grille(n=n, p=p, seed=seed)
random.seed(seed)
(x,y) = (random.randint(0, n-1), random.randint(0, n-1))
print_grille(G, pos=(x, y), label="Grille initiale (R = robot) :")
max_steps = 30
step = 0
actions = []
# Boucle d'exécution 
while not tout_propre(G) and step < max_steps: 
    # Le robot choisit une action 
    a = choisir_action(G, x, y, step)
    # Le robot applique l'action 
    x, y, a_eff = appliquer_action(G, x, y, a)

    # Enregistrer l'action 
    actions.append(a_eff)
    step += 1 
# Calcul de la performance 
propres = sum(1 for i in range(n) for j in range(n) if G[i][j] == 0)
taux = 100.0 * propres / (n*n)
# Affichage des résultats 
print_grille(G, (x, y), label="Grille finale :") 
print({ 
    "cases_propres": propres,  
    "sur": n*n,  
    "taux_%": round(taux, 1),  
    "nb_actions": len(actions)
}) 
print("\nActions effectuées :", actions) 
 

Grille initiale (R = robot) :
1 0 R
1 1 1
0 0 1

Grille finale :
R 0 0
0 0 0
0 0 0

{'cases_propres': 9, 'sur': 9, 'taux_%': 100.0, 'nb_actions': 17}

Actions effectuées : ['BAS', 'ASPIRER', 'GAUCHE', 'ASPIRER', 'DROITE', 'BAS', 'ASPIRER', 'HAUT', 'BAS', 'GAUCHE', 'GAUCHE', 'HAUT', 'ASPIRER', 'BAS', 'HAUT', 'HAUT', 'ASPIRER']
